In [1]:
from datetime import datetime
import pytz
from pytz import timezone
from pytz import common_timezones
from matplotlib import cm
import plotly.figure_factory as ff
import numpy as np

In [18]:
def get_Gantt_Params(Schedule_Text,in_fmt='%Y-%m-%d %H:%M:%S',out_tz='UTC',debug=False):

    # How many scopes are listed
    List_Scopes = set([x.split(',')[0] for x in Schedule_Text.split(';') if x])
    no_scopes   = len(List_Scopes)
    row_number  = no_scopes-1
    
    # How many Resources are listed (and hence how many colors needed)
    List_Rsrcs   = set([x.split(',')[4] for x in Schedule_Text.split(';') if x])
    no_Rsrcs     = len(List_Rsrcs)
    color_number = -1
    intervals    = np.linspace(0, 1, no_Rsrcs)
    cm_          = [cm.rainbow(x) for x in intervals]
    
    # Set output timezone and format
    output_tz   = timezone(out_tz)
    out_fmt     = '%Y-%m-%d %H:%M:%S'
    
    # Initiate outputs
    Gantt_Entries  = []
    Gantt_Annots   = []
    Track_Scopes   = set([])
    Track_Rsrcs    = set([])
    RowInfo_Dict   = {}
    ColorInfo_Dict = {}

    for line in Schedule_Text.split(';'):
        # Catch empty lines
        if len(line) > 0:
            # Get Entry info
            scope,start,end,in_tz,rsrc,label = line.split(',')
            input_tz                         = timezone(in_tz)

            # Figure out what row is this line plotted on gantt chart
            if scope not in Track_Scopes:
                RowInfo_Dict[scope]=row_number
                row_number -= 1
                Track_Scopes.add(scope)
                
            # Figure out if a new color is needed, if yes, then add. 
            if rsrc not in Track_Rsrcs:
                color_number += 1
                ColorInfo_Dict[rsrc]=cm2rgbstring(cm_[color_number])
                if debug:
                    print('DEBUG: COLORINFO_DICT added: ',ColorInfo_Dict[rsrc])
                Track_Rsrcs.add(rsrc)
                
            # Get inverse of the color for this rsrc
            inv_c=invert_color(ColorInfo_Dict[rsrc],debug=debug)

            # Convert times to datetime with right input timezone information
            start_pytz = datetime.strptime(start,in_fmt).replace(tzinfo=input_tz) 
            end_pytz   = datetime.strptime(end,in_fmt).replace(tzinfo=input_tz) 

            # Convert times to ouptut timezone
            start_out   = start_pytz.astimezone(output_tz)
            end_out     = end_pytz.astimezone(output_tz)
            midtime_out = start_out+(end_out-start_out)/2

            # Get times to string for gantt dictionary
            s = start_out.strftime('%Y-%m-%d %H:%M:%S')
            e = end_out.strftime('%Y-%m-%d %H:%M:%S')
            m = midtime_out.strftime('%Y-%m-%d %H:%M:%S')

            # Get Gantt Dictionary
            d    = dict(Task=scope, 
                        Start=s, 
                        Finish=e, 
                        Resource=rsrc)
            d_a  = dict(x=m,
                        y=RowInfo_Dict[scope],
                        text=label, 
                        showarrow=False, 
                        font=dict(color=inv_c)
                       )
            if debug:
                print('DEBUG: Inverse Color: ',inv_c)
            Gantt_Entries.append(d)
            Gantt_Annots.append(d_a)
             
    return Gantt_Entries,ColorInfo_Dict,Gantt_Annots,List_Scopes

def cm2rgbstring(c):
    rgbstring=f'rgb( {int(c[0]*255)},{int(c[1]*255)},{int(c[2]*255)})'
    return rgbstring

def invert_color(c,debug=False):
    if debug:
        print('DEBUGGING invert_color')
        print(c)
    a=[int(x) for x in c.split('(')[1].split(')')[0].split(',')]
    b = [255-x for x in a]
    if debug:
        print(b)
    inverted_color = f'rgb( {b[0]},{b[1]},{b[2]})'
    return inverted_color

In [23]:
def Plot_Schedule(Schedule_Text,in_fmt='%Y-%m-%d %H:%M:%S',out_tz='UTC',debug=False):
    Gantt_Entries,colors,Gantt_Annots,List_Scopes = get_Gantt_Params(Schedule_Text,
                                                                     in_fmt='%Y-%m-%d %H:%M:%S',
                                                                     out_tz=out_tz,
                                                                     debug=debug)
    
    # Overwite colors output from get_Gantt_Params
    colors = dict(Neutrinos  = 'rgb(  0,   0,   0)',
                  Gamma      = 'rgb(  0,   0, 150)',
                  XRay       = 'rgb(  0,   0, 250)',
                  UVXRay     = 'rgb(  0,   50, 250)',
                  UV         = 'rgb(  0, 100, 250)',
                  Vis        = 'rgb(  0, 200,   0)',
                  Radio      = 'rgb(250,   0,   0)')
    
    if debug:
        print(colors)

    No_Lines = len(List_Scopes)+1

    fig = ff.create_gantt(Gantt_Entries, colors=colors, index_col='Resource', title='DWF Schedule',
                          show_colorbar=True, showgrid_x=True, showgrid_y=True, group_tasks=True,
                          bar_width=0.4,height=50*No_Lines) 
    fig['layout']['annotations'] = Gantt_Annots
    fig.show()

In [25]:
# Table for diff datetime formats
# https://www.journaldev.com/23365/python-string-to-datetime-strptime
# Do it like this: '%Y-%m-%d %H:%M:%S' = '2016-01-01 11:30:00'
fmt = '%Y-%m-%d %H:%M:%S'

# DECam Schedule from Jeff Cooke's email 
# Parkes Schedule from https://www.parkes.atnf.csiro.au/observing/schedules/current/PK.pdf 
# Parks Scheule also from Vivek Gupta's email subject: "Swapping of Parkes observing time for P1050 - P1066 - BL"
# ASTROSAT Schedule from email chain subject: "Your Astrosat proposal A09_107 has been accepted for cycle A09"
Schedule_Text = "DECam,2020-09-13 04:30:00,2020-09-13 05:00:00,UTC,Vis,N;"\
                "DECam,2020-09-14 04:30:00,2020-09-14 05:00:00,UTC,Vis,N;"\
                "DECam,2020-09-15 04:30:00,2020-09-15 05:00:00,UTC,Vis,N;"\
                "DECam,2020-09-17 04:30:00,2020-09-17 05:00:00,UTC,Vis,N;"\
                "DECam,2020-09-18 04:30:00,2020-09-18 05:00:00,UTC,Vis,N;"\
                "DECam,2020-09-19 04:30:00,2020-09-19 05:00:00,UTC,Vis,N;"\
                ""\
                "DECam,2020-09-13 05:05:05,2020-09-13 06:30:00,UTC,Vis,F18;"\
                "DECam,2020-09-14 05:05:00,2020-09-14 06:30:00,UTC,Vis,F18;"\
                "DECam,2020-09-15 05:05:00,2020-09-15 06:30:00,UTC,Vis,F18;"\
                "DECam,2020-09-17 05:05:00,2020-09-17 06:30:00,UTC,Vis,F18;"\
                "DECam,2020-09-18 05:05:00,2020-09-18 06:30:00,UTC,Vis,F18;"\
                "DECam,2020-09-19 05:05:00,2020-09-19 06:30:00,UTC,Vis,F18;"\
                ""\
                "DECam,2020-09-13 06:31:00,2020-09-13 08:00:00,UTC,Vis,F19;"\
                "DECam,2020-09-14 06:31:00,2020-09-14 08:00:00,UTC,Vis,F19;"\
                "DECam,2020-09-15 06:31:00,2020-09-15 08:00:00,UTC,Vis,F19;"\
                "DECam,2020-09-17 06:31:00,2020-09-17 08:00:00,UTC,Vis,F19;"\
                "DECam,2020-09-18 06:31:00,2020-09-18 08:00:00,UTC,Vis,F19;"\
                "DECam,2020-09-19 06:31:00,2020-09-19 08:00:00,UTC,Vis,F19;"\
                ""\
                "DECam,2020-09-13 08:01:00,2020-09-13 09:00:00,UTC,Vis,E;"\
                "DECam,2020-09-14 08:01:00,2020-09-14 09:00:00,UTC,Vis,E;"\
                "DECam,2020-09-15 08:01:00,2020-09-15 09:00:00,UTC,Vis,E;"\
                "DECam,2020-09-17 08:01:00,2020-09-17 09:00:00,UTC,Vis,E;"\
                "DECam,2020-09-18 08:01:00,2020-09-18 09:00:00,UTC,Vis,E;"\
                "DECam,2020-09-19 08:01:00,2020-09-19 09:00:00,UTC,Vis,E;"\
                ""\
                "Parkes,2020-09-13 04:00:00,2020-09-13 10:00:00,UTC,Radio,;"\
                "Parkes,2020-09-14 04:00:00,2020-09-14 10:00:00,UTC,Radio,;"\
                "Parkes,2020-09-15 04:00:00,2020-09-15 10:00:00,UTC,Radio,;"\
                "Parkes,2020-09-17 04:00:00,2020-09-17 10:00:00,UTC,Radio,;"\
                "Parkes,2020-09-18 04:00:00,2020-09-18 10:00:00,UTC,Radio,;"\
                "Parkes,2020-09-19 04:00:00,2020-09-19 10:00:00,UTC,Radio,;"\
                ""\
                "ASTROSAT,2020-09-14 06:00:00,2020-09-14 08:00:00,UTC,UVXRay,;"\
                "ASTROSAT,2020-09-15 06:00:00,2020-09-15 08:00:00,UTC,UVXRay,;"\
                "ASTROSAT,2020-09-18 06:00:00,2020-09-18 08:00:00,UTC,UVXRay,;"\
                "AAT-Geha,2020-09-12 19:19:00,2020-09-13 04:32:00,Australia/Sydney,Vis,;"\
                "AAT-Geha,2020-09-13 19:19:00,2020-09-14 04:32:00,Australia/Sydney,Vis,;"\
                "AAT-Geha,2020-09-14 19:19:00,2020-09-15 04:32:00,Australia/Sydney,Vis,;"\
                "AAT-Geha,2020-09-15 19:19:00,2020-09-16 04:32:00,Australia/Sydney,Vis,;"\
                "AAT-Geha,2020-09-16 19:19:00,2020-09-17 04:32:00,Australia/Sydney,Vis,;"\
                "AAT-Geha,2020-09-17 19:19:00,2020-09-18 04:32:00,Australia/Sydney,Vis,;"\
                "AAT-Geha,2020-09-18 19:19:00,2020-09-19 04:32:00,Australia/Sydney,Vis,;"\
                "AAT-Geha,2020-09-19 19:19:00,2020-09-20 04:32:00,Australia/Sydney,Vis,;"\
                "AAT-Geha,2020-09-20 19:19:00,2020-09-21 04:32:00,Australia/Sydney,Vis,;"\
                "AAT,2020-09-21 19:19:00,2020-09-22 04:32:00,Australia/Sydney,Vis,;"\
                "AAT,2020-09-22 19:19:00,2020-09-23 04:32:00,Australia/Sydney,Vis,;"\
                "AAT,2020-09-23 19:19:00,2020-09-24 04:32:00,Australia/Sydney,Vis,;"\
                "AAT,2020-09-24 19:19:00,2020-09-25 04:32:00,Australia/Sydney,Vis,;"\
                "WiFeS,2020-09-14 19:19:00,2020-09-15 04:32:00,Australia/Sydney,Vis,;"\
                "WiFeS,2020-09-15 19:19:00,2020-09-16 04:32:00,Australia/Sydney,Vis,;"\
                "WiFeS,2020-09-16 19:19:00,2020-09-17 04:32:00,Australia/Sydney,Vis,;"\
                "WiFeS,2020-09-17 19:19:00,2020-09-18 04:32:00,Australia/Sydney,Vis,;"\
                "WiFeS,2020-09-18 19:19:00,2020-09-19 04:32:00,Australia/Sydney,Vis,;"\
                "WiFeS,2020-09-19 19:19:00,2020-09-20 04:32:00,Australia/Sydney,Vis,;"\

Plot_Schedule(Schedule_Text,in_fmt='%Y-%m-%d %H:%M:%S',out_tz='UTC')
Plot_Schedule(Schedule_Text,in_fmt='%Y-%m-%d %H:%M:%S',out_tz='Australia/Sydney')
print(' N  =NGC6744 \n F18=FRB181112 \n F19=FRB190711 \n E  =Elias')

 N  =NGC6744 
 F18=FRB181112 
 F19=FRB190711 
 E  =Elias
